In [1]:
import os
import json

In [2]:
pgx_genes=[]
for line in open("/scratch/NagarajLab_BG/indigen_proj/8_jointgenotypge/sentieon_combined_vcf/Tiwi_187_hardeep38_SN32/DB_PGKB_genes.txt"):
    line=line.strip()
    if line not in pgx_genes:
        pgx_genes.append(line)
print(len(pgx_genes))

921


In [13]:
vidstore=[]
for line in open("Tiwi_cohort_191_hg19_sorted_filtered_annovar.hg19_multianno.txt"):
    line=line.strip()
    if line.startswith("Chr"):
        continue
    annot=line.split("\t")
    a1=27
    a2=31
    a3=44
    vid=annot[0]+":"+annot[1]+":"+annot[3]+":"+annot[4]    
    flag=0
    if annot[a1]=="D":
        flag=flag+1
    elif annot[a2]=="D":
        flag=flag+1
    elif annot[a2]=="P":
        flag=flag+0.5
    try:
        phred=float(annot[a3])
        if phred>=15:
            flag=flag+1
        elif phred>=10:
            flag=flag+0.5
    except:
        pass
    if flag>=2:
        vidstore.append(vid)
    else:
        continue

In [14]:
ofile=open("Tiwi_pgx_genes_delvar.vcf","w")
for line in open("Tiwi_cohort_191_hg19_sorted_filtered_annovar.hg19_multianno_PGX.vcf"):
    line=line.strip()
    if line.startswith("#"):
        ofile.write(line+"\n")
        continue
    cols=line.split("\t")
    if cols[2].strip() in vidstore:
        ofile.write(line+"\n")
ofile.close()

In [15]:
pgx1b=[]
for line in open("/home/n10357050/ihbi_tiwi_pgx/PhramGKB/new_analsyis_pgx1b/posid_rev"):
    line=line.strip()
    if line not in pgx1b:
        pgx1b.append(line)
print(len(pgx1b))

131


In [16]:
ofile=open("tiwi_pgx1a1b.vcf","w")
for line in open("Tiwi_cohort_191_hg19_sorted_filtered_annovar.hg19_multianno_PGX.vcf"):
    line=line.strip()
    if line.startswith("#"):
        ofile.write(line+"\n")
        continue
    cols=line.split("\t")
    posid=":".join(cols[2].split(":")[0:2])
    if posid in pgx1b:
        ofile.write(line+"\n")
ofile.close()

In [18]:
with open("/scratch/NagarajLab_BG/indigen_proj/8_jointgenotypge/sentieon_combined_vcf/Tiwi_187_hardeep38_SN32/pgx_starallele_dictionaryv8.json") as jf:
    stardictpos=json.load(jf)
allstarsnps=[]
for s in stardictpos:
    for snp in stardictpos[s]:
        allstarsnps.append(snp)
uniquestarsnps=list(set(allstarsnps))

In [19]:
len(uniquestarsnps)

1186

In [20]:
ofile=open("tiwi_pgx_allstar.vcf","w")
for line in open("Tiwi_cohort_191_hg19_sorted_filtered_annovar.hg19_multianno_PGX.vcf"):
    line=line.strip()
    if line.startswith("#"):
        ofile.write(line+"\n")
        continue
    cols=line.split("\t")
    posid=cols[2]
    if posid in uniquestarsnps:
        ofile.write(line+"\n")
ofile.close()

In [1]:
import vcf

In [2]:
vcf_reader = vcf.Reader(filename='Tiwi_cohort_191_hg19_sorted_filtered.vcf.gz', compressed=True)
vcf_writer = vcf.Writer(open('Tiwi_cohort_191_hg19_sorted_filtered_BER_v2.vcf', 'w'), vcf_reader)
for rec in vcf_reader:
    t1count=0
    t2count=0
    t3count=0
    for sam in rec.samples:
        sname=sam.sample
        if sname.startswith("TIWI") and "1" in sam["GT"]:
            t1count=t1count+1
        elif sname.startswith("FD") and "1" in sam["GT"]:
            t2count=t2count+1
        elif "1" in sam["GT"]:
            t3count=t3count+1
    if t1count!=0:
        t1=1
    else:
        t1=0
    if t2count!=0:
        t2=1
    else:
        t2=0
    if t3count!=0:
        t3=1
    else:
        t3=0
    tsum=t1+t2+t3
    if tsum>=2:
        vcf_writer.write_record(rec)